- 네이버 영화 평점: https://movie.naver.com/movie/point/af/list.nhn
- 훈련 데이터: ratings_train.txt
- Test 데이터: ratings_test.txt
- 데이터: https://github.com/e9t/nsmc
    - 리뷰: document
    - 긍부정: label - 0:부정, 1:긍정

## 훈련/테스트 데이터 셋 읽기

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('ratings_train.txt', sep='\t')

In [3]:
test_df = pd.read_csv('ratings_test.txt', sep='\t')

## 데이터셋 확인

In [4]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [5]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
id          150000 non-null int64
document    149995 non-null object
label       150000 non-null int64
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
id          50000 non-null int64
document    49997 non-null object
label       50000 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [9]:
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [11]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [12]:
# 결측치 제거
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [13]:
train_df.isna().sum(), test_df.isna().sum()

(id          0
 document    0
 label       0
 dtype: int64, id          0
 document    0
 label       0
 dtype: int64)

In [15]:
# label의 분포
train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [16]:
test_df['label'].value_counts()

1    25171
0    24826
Name: label, dtype: int64

## 텍스트 전처리

In [21]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [18]:
import re, string
from konlpy.tag import Okt

In [22]:
def text_preprocessing(document):
    okt = Okt()
    # 구두점 제거
    pattern = '[{}]'.format(string.punctuation)
    document = re.sub(pattern, ' ', document)
    #토큰화
    token = okt.morphs(document, stem=True, norm=True) 
#     token_nouns = okt.nouns(document) #명사기준 토큰화
    return ' '.join(token)

In [24]:
train_df['document'][0]

'아 더빙.. 진짜 짜증나네요 목소리'

In [25]:
text_preprocessing(train_df['document'][0])

'아 더빙 진짜 짜증나다 목소리'

In [27]:
# 전체 document에 text_preprocessing 함수 적용
X_train = train_df['document'].apply(text_preprocessing)
X_train[:3]

0                    아 더빙 진짜 짜증나다 목소리
1    흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다
2             너 무재 밓었 다그 래서 보다 추천 한 다
Name: document, dtype: object

In [28]:
y_train = train_df['label']

In [29]:
X_test = test_df['document'].apply(text_preprocessing)

In [30]:
y_test = test_df['label']

## Feature Vectorization

In [ ]:
아 더빙 진짜 짜증나다 목솔

아 = uni-gram(1-gram)
아 더빙 = bi-gram(2-gram)
아 더빙 진짜 = tri-gram(3-gram)
더빙
더빙 진짜 
더빙 진짜 짜증나다


In [31]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [33]:
#CounterVectorizer : 단순 빈도수
cv = CountVectorizer(min_df=10)
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [34]:
X_train_cv.shape

(149995, 8360)

In [35]:
tfidf = TfidfVectorizer(min_df=10)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [36]:
X_train_tfidf.shape

(149995, 8360)

In [40]:
X_train_tfidf

<149995x8360 sparse matrix of type '<class 'numpy.float64'>'
	with 1157210 stored elements in Compressed Sparse Row format>

In [ ]:
#X_train_tfidf.toarray()

## 긍부정 예측 머신러닝 모델링

In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
lg_clf = LogisticRegression()
lg_clf.fit(X_train_cv, y_train)

C:\Users\Playdata\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
from sklearn.metrics import accuracy_score

In [44]:
pred_train = lg_clf.predict(X_train_cv)
accuracy_score(y_train, pred_train)

0.8533017767258909

In [45]:
pred_test = lg_clf.predict(X_test_cv)
accuracy_score(y_test, pred_test)

0.8256495389723384

In [46]:
# Tfidf 
lg_clf = LogisticRegression()
lg_clf.fit(X_train_tfidf, y_train)

C:\Users\Playdata\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [47]:
pred_train = lg_clf.predict(X_train_tfidf)
accuracy_score(y_train, pred_train)

0.8503216773892464

In [48]:
pred_test = lg_clf.predict(X_test_tfidf)
accuracy_score(y_test, pred_test)

0.8291097465847951

In [51]:
new_review=[
"너무좋다.단지 조금 아쉬운 점은 식기가 너무 새것인 것과 고모를 보는 시각이 아쉽다. 일본것은 실망했는데 이편은 배우.스토리.연출 모든것이 좋다",
"정말로 재미있었어요",
'화폭발이 아닌 핵폭팔로 바꾸어 생각해본다면 보다 현실적영화가 되었을 것이다. 핵" 문제는 남과북 이런다고 해결될문제는 아닌것같다. 핵"을 두고 생각하면 누가 적이고 우군인지를 알것이다. "북은 절때 우군이 될~수없다 그들의 주적은 남한 일 것이다. 선동에 포섭될 학력인가?']


In [52]:
new_review = [text_preprocessing(review)  for review in new_review]
new_review

['너무 좋다 단지 조금 아쉽다 점 은 식기 가 너무 새다 것 과 고모 를 보다 시각 이 아쉽다 일 본 것 은 실망하다 이편 은 배우 스토리 연출 모든 것 이 좋다',
 '정말로 재미있다',
 '화 폭발 이 아니다 핵 폭팔 로 바꾸다 생각 해보다 보다 현실 적 영화 가 되어다 것 이다 핵 문제 는 남 과 북 이렇다 해결 되다 문제 는 아니다 같다 핵 을 두다 생각 하다 누가 적 이고 우군 인지 를 알다 북 은 절때 우군 이 되다 수 없다 그 들 의 주적 은 남한 일 것 이다 선동 에 포섭 되다 학력 인가']

In [55]:
new_X = tfidf.transform(new_review)

In [56]:
lg_clf.predict(new_X)

array([1, 1, 0], dtype=int64)